In [1]:
import pandas as pd
from IPython.display import Image #type: ignore

In [2]:
display(Image(url="Data/Tech_challenge_2.png", width=800, height=500))

- Carregar base

In [3]:
df = pd.read_csv('Data/Dados Históricos - Ibovespa.csv')
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,09.03.2023,105.071,106.540,106.724,105.053,"19,17M","-1,38%"
1,08.03.2023,106.540,104.228,106.721,104.228,"15,90M","2,22%"
2,07.03.2023,104.228,104.700,105.179,103.480,"12,58M","-0,45%"
3,06.03.2023,104.700,103.865,105.171,103.170,"14,28M","0,80%"
4,03.03.2023,103.866,103.326,104.440,103.323,"13,10M","0,52%"


- Tratar valores

In [4]:
df.isnull().sum()

Data        0
Último      0
Abertura    0
Máxima      0
Mínima      0
Vol.        1
Var%        0
dtype: int64

In [5]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4999 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      4999 non-null   object 
 1   Último    4999 non-null   float64
 2   Abertura  4999 non-null   float64
 3   Máxima    4999 non-null   float64
 4   Mínima    4999 non-null   float64
 5   Vol.      4999 non-null   object 
 6   Var%      4999 non-null   object 
dtypes: float64(4), object(3)
memory usage: 312.4+ KB


In [6]:
def convert_volume(volume_str):
    volume_str = volume_str.replace(',', '.')
    if 'M' in volume_str:
        return int(float(volume_str.replace('M', '')) * 1000000)
    elif 'K' in volume_str:
        return int(float(volume_str.replace('K', '')) * 1000)
    else:
        return int(volume_str)

df['Vol.'] = df['Vol.'].apply(convert_volume)
df['Var%'] = df['Var%'].str.replace('%', '').str.replace(',', '.').astype(float)
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,09.03.2023,105.071,106.540,106.724,105.053,19170000,-1.38
1,08.03.2023,106.540,104.228,106.721,104.228,15900000,2.22
2,07.03.2023,104.228,104.700,105.179,103.480,12580000,-0.45
3,06.03.2023,104.700,103.865,105.171,103.170,14280000,0.80
4,03.03.2023,103.866,103.326,104.440,103.323,13100000,0.52


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4999 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      4999 non-null   object 
 1   Último    4999 non-null   float64
 2   Abertura  4999 non-null   float64
 3   Máxima    4999 non-null   float64
 4   Mínima    4999 non-null   float64
 5   Vol.      4999 non-null   int64  
 6   Var%      4999 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 312.4+ KB


- Ajustar campo **Data** e **index**

In [8]:
df.index = pd.to_datetime(df.Data, format='%d.%m.%Y') #type: ignore
df.drop('Data', inplace=True, axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4999 entries, 2023-03-09 to 2003-01-02
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Último    4999 non-null   float64
 1   Abertura  4999 non-null   float64
 2   Máxima    4999 non-null   float64
 3   Mínima    4999 non-null   float64
 4   Vol.      4999 non-null   int64  
 5   Var%      4999 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 273.4 KB


- análise

In [10]:
df.head()

,Último,Abertura,Máxima,Mínima,Vol.,Var%
Data,,,,,,
2023-03-09,105.071,106.540,106.724,105.053,19170000,-1.38
2023-03-08,106.540,104.228,106.721,104.228,15900000,2.22
2023-03-07,104.228,104.700,105.179,103.480,12580000,-0.45
2023-03-06,104.700,103.865,105.171,103.170,14280000,0.80
2023-03-03,103.866,103.326,104.440,103.323,13100000,0.52
